In [5]:
from requests import get # la vamos a usar para solicitar la página mediante su URL 
from bs4 import BeautifulSoup # biblioteca para web scraping
import pandas as pd
import string                              
from nltk.corpus import stopwords # para remover palabras que no agregan significado          
from nltk.stem import PorterStemmer # para llevar las palabras a su forma raíz        
from nltk.tokenize import TweetTokenizer # para transformar cada palabra en un token
from IPython.display import clear_output
from collections import defaultdict
import numpy as np

In [6]:
import pickle
def save_obj(obj, name):
       with open(name + '.pkl', 'wb') as f:
           pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
       with open(name + '.pkl', 'rb') as f:
           return pickle.load(f)

In [ ]:
class Textos:
    clasesDefault = 5
    listaDataset = []
    def __init__(self, clases, webs):
        import pandas as pd
        from requests import get      # la vamos a usar para solicitar la página mediante su URL 
        from bs4 import BeautifulSoup # biblioteca para web scraping
        import string                              
        from nltk.corpus import stopwords   # para remover palabras que no agregan significado          
        from nltk.stem import PorterStemmer # para llevar las palabras a su forma raíz        
        from nltk.tokenize import TweetTokenizer # para transformar cada palabra en un token
        from IPython.display import clear_output
        from collections import defaultdict
        import numpy as np
        self.clases = clases
        listaDataset = []
        listaDataset.append('TextoCrudo')
        listaDataset.append('TextoLimpio')
        for i in range(1,clases +1):
            listaDataset.append('Clase' + str(i))
        listaDataset.append('Predicción')
        listaDataset.append('CLASE')
        self.dataFrame = pd.DataFrame(columns = listaDataset)
        self.webs = pd.ExcelFile(webs)
        df = self.webs.parse(self.webs.sheet_names[0])
        self.__clases_df = set(df["CÓDIGO"].value_counts().index)
        for i in df.index: 
            print(f"Creando Dataframe {i} de {len(df.index)}")   
            print("i", i)
            url = df["URL"][i]
            req = get(url)
            statusCode = req.status_code
            html = BeautifulSoup(req.text, "html.parser")
            if statusCode == 200:
                paragraphs = html.select("p")
                cont = 0
                for para in paragraphs:
                    if len(para.text) < 15:
                        continue
                    cont += 1 
                    parrafo = ""
                    print("parrafo",parrafo) 
                    print("i",i) 
                    print("df['CÓDIGO'][i]",df["CÓDIGO"][i]) 
                    print('df["NOMBRE ARCHIVO"][i]',df["NOMBRE ARCHIVO"][i])
                    print("cont",cont) 
                    guardarComo = str(df["CÓDIGO"][i]) + "/"  + df["NOMBRE ARCHIVO"][i] + str(cont)
                    print("guardarComo",guardarComo)  
                    parrafo = open(guardarComo,"w",encoding = "utf-8") 
                    parrafo.write(para.text)
                    self.dataFrame = self.dataFrame.append({'TextoCrudo': para.text, 'CLASE':df["CÓDIGO"][i]}, ignore_index=True)
                    parrafo.close()
                clear_output( wait = True )
                 
        
    @property
    def clases(self):
        return self.__clases
    
    @clases.setter
    def clases(self, clases):
        clasesDefault = 5
        if clases < 11:
            self.__clases = clases
        else:
            self.__clases = clasesDefault           
        return None      
    
   
    @property
    def dataFrame(self):
        return self.__dataFrame
    
    @dataFrame.setter
    def dataFrame(self, cambios):
        self.__dataFrame = cambios
        return None      
    
    @property
    def webs(self):
        return self.__webs
    
    @webs.setter
    def webs(self, webs):
        self.__webs = webs
        return None         
   

In [ ]:
clases = 5
webs = "Webs.xls"

In [ ]:
textos = Textos(clases, webs)

In [ ]:
textos.clases

In [ ]:
textos.dataFrame

In [ ]:
textos.dataFrame.to_csv("Historia.csv")

In [ ]:
import pandas as pd
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

In [ ]:
textos.iloc[4029,0]

In [ ]:
for i in textos.index:
    textos["LTCrudo"][i] = len(textos["TextoCrudo"][i])
textos

In [ ]:
textos.describe()

In [ ]:
for i in textos.index:
    textos["TextoLimpio"][i] = limpiar(textos["TextoCrudo"][i])
textos        

In [ ]:
def limpiar(textoCrudo):
    for i in range(100):
        textoCrudo = textoCrudo.replace("[" + str(i) + "]", "")
    for i in ["a","b","c"]:
        textoCrudo = textoCrudo.replace("[" + i + "]", "")
    stopwords_english = stopwords.words('english')
    stop        = stopwords.words('english')
    tokenizer = TweetTokenizer(preserve_case = False, 
                           strip_handles = True,
                           reduce_len    = True)
    stemmer = PorterStemmer()    
    texto_tokens = tokenizer.tokenize(textoCrudo)    
    for i in range(len(texto_tokens.copy())):
        if i > len(texto_tokens) - 1:
            break
        if i !=0:
            if texto_tokens[i] == "bc" and texto_tokens[i - 1].isnumeric():
                texto_tokens[i] = texto_tokens[i - 1] + texto_tokens[i]     
                texto_tokens.pop(i - 1)         
    texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation]  
    texto_stem   = [stemmer.stem(word) for word in texto_clean] 
    return texto_stem        

In [ ]:
crudo = "10 bc bcc Hola 10 bcc chau 5 bc  555 bc bcc 10"
limpio = limpiar(crudo)
print(type(limpio))
limpio

In [ ]:
posicionBC = 2
limpio[posicionBC - 1] + limpio[posicionBC] 

In [ ]:
textos["LTLimpio"] = None
textos

In [ ]:
for i in textos.index:
    textos["LTCrudo"][i] = len(textos["TextoCrudo"][i])
    textos["LTLimpio"][i] = len(textos["TextoLimpio"][i])
textos

In [ ]:
type(textos["TextoLimpio"][0])

In [ ]:
len(textos["TextoLimpio"][0])

In [ ]:
textos

In [ ]:
textos["TextoLimpio"][6063]

In [ ]:
conta = 0
for i in textos.index:
    if len(textos["TextoLimpio"][i]) < 10:
        conta += 1 
        print(textos["TextoLimpio"][i]) 
print(conta)        

In [ ]:
textos = textos[textos["LTLimpio"] > 7]
textos

In [7]:
textos.to_csv("Historia.csv")

NameError: name 'textos' is not defined

In [9]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",NaN,NaN,NaN,NaN,NaN,NaN,1,757,78
1,"During his youth, Alexander was tutored by Ari...","['youth', 'alexand', 'tutor', 'aristotl', 'age...",NaN,NaN,NaN,NaN,NaN,NaN,1,821,74
2,"Alexander endeavoured to reach the ""ends of th...","['alexand', 'endeavour', 'reach', 'end', 'worl...",NaN,NaN,NaN,NaN,NaN,NaN,1,624,58
3,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",NaN,NaN,NaN,NaN,NaN,NaN,1,1007,87
4,"Alexander was born in Pella, the capital of th...","['alexand', 'born', 'pella', 'capit', 'kingdom...",NaN,NaN,NaN,NaN,NaN,NaN,1,488,46
...,...,...,...,...,...,...,...,...,...,...,...
5861,Some agriculturalists also regularly hunt and ...,"['agriculturalist', 'also', 'regularli', 'hunt...",NaN,NaN,NaN,NaN,NaN,NaN,5,592,54
5862,There are nevertheless a number of contemporar...,"['nevertheless', 'number', 'contemporari', 'hu...",NaN,NaN,NaN,NaN,NaN,NaN,5,1011,86
5863,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",NaN,NaN,NaN,NaN,NaN,NaN,5,549,54
5864,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'would', 'eventu', 'flourish...",NaN,NaN,NaN,NaN,NaN,NaN,5,724,71


In [10]:
clases_df = {1, 2, 3 , 4 , 5}
clases_df

{1, 2, 3, 4, 5}

In [ ]:
frecuencias = defaultdict(int)
vocabulario = set([x for m in textos.index for x in textos.loc[m].TextoLimpio ])                

for i in textos.index:
    for palabra in textos.loc[i].TextoLimpio:
        frecuencias[(palabra, textos.loc[i].CLASE)] += 1
        clear_output( wait = True )
        print(f"Creando Frecuencias {i} de {len(textos)}")

for i in clases_df:
    textos[f"Clase{i}"] = textos.TextoLimpio.apply(lambda x : etiquetas(x,frecuencias,clases_df)[0,i])
print("dataset terminado")
print(textos.describe())

In [ ]:
frecuencias = defaultdict(int)
vocabulario = set([x for m in textos.index for x in textos.loc[m].TextoLimpio ])   

In [ ]:
len(vocabulario)

In [ ]:
vocabulario

In [14]:
def limpiarWord(word):
    word = word[:-1]
    return word

In [ ]:
listaPalabras = []
cont = 0
for x in textos.index:
    cont += 1
    if cont > 10000000:
        break
    for word in textos["TextoLimpio"][x][:-1].split(","):
        word = word[2:-1]
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        listaPalabras.append(word)    
vocabulario = set(listaPalabras)
print()
print("largo Vocabulario",len(vocabulario))
print()
print(vocabulario)

In [ ]:
for word in vocabulario:
    print(word)

In [ ]:
save_obj(vocabulario, "Vocabulario")

In [ ]:
vocabulario = load_obj("Vocabulario")

In [15]:
frecuencias = defaultdict(int)
clases = 5
cont = 0
for i in textos.index:
    cont += 1
    if cont > 10000000:
        break
    for word in textos["TextoLimpio"][i][:-1].split(","):
        word = word[2:-1]
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        frecuencias[(word, textos["CLASE"][i])] = frecuencias.get((word, textos["CLASE"][i]), 0) + 1    
    clear_output( wait = True )
    print(f"Creando Frecuencias {i} de {len(textos.index)}")      
frecuencias

Creando Frecuencias 5865 de 5866


defaultdict(int,
            {('alexand', 1): 297,
             ('iii', 1): 57,
             ('macedon', 1): 18,
             ('greek', 1): 271,
             ('αλέξανδρος', 1): 1,
             ('γʹ', 1): 1,
             ('ὁ', 1): 2,
             ('μακεδών', 1): 1,
             ('aléxandro', 1): 1,
             ('ho', 1): 2,
             ('makedȏn', 1): 1,
             ('20/21', 1): 1,
             ('juli', 1): 7,
             ('356bc', 1): 3,
             ('–', 1): 212,
             ('10/11', 1): 1,
             ('june', 1): 3,
             ('323bc', 1): 4,
             ('commonli', 1): 16,
             ('known', 1): 166,
             ('great', 1): 126,
             ('μέγας', 1): 1,
             ('méga', 1): 1,
             ('king', 1): 240,
             ('basileu', 1): 3,
             ('ancient', 1): 219,
             ('kingdom', 1): 128,
             ('member', 1): 28,
             ('argead', 1): 1,
             ('dynasti', 1): 83,
             ('born', 1): 15,
             ('pella',

In [16]:
print("Largo frecuencias:",len(frecuencias))

Largo frecuencias: 47920


In [17]:
for i in frecuencias.keys():
    if frecuencias[i] > 200:
        print(i,frecuencias[i])

('alexand', 1) 297
('greek', 1) 271
('–', 1) 212
('king', 1) 240
('ancient', 1) 219
('one', 1) 229
('empir', 1) 264
('use', 1) 326
('power', 1) 217
('citi', 1) 243
('war', 1) 260
('cultur', 1) 204
('centuri', 1) 284
('also', 1) 320
('first', 1) 226
('rome', 1) 298
('caesar', 1) 272
('roman', 1) 729
('carthag', 1) 203
('parthian', 1) 232
('centuri', 2) 539
('roman', 2) 263
('empir', 2) 414
('use', 2) 246
('time', 2) 223
('byzantin', 2) 360
('europ', 2) 222
('vike', 2) 310
('first', 2) 221
('war', 2) 227
('also', 2) 290
('christian', 2) 298
('king', 2) 406
('england', 2) 221
('one', 2) 204
('muslim', 2) 228
('land', 2) 213
('power', 2) 209
('lombard', 2) 201
('state', 2) 202
('norman', 2) 221
('english', 2) 224
('crusad', 2) 309
('william', 2) 293
('centuri', 3) 234
('european', 3) 215
('also', 3) 261
('spain', 3) 245
('first', 3) 297
('new', 3) 246
('henri', 3) 253
('year', 3) 225
('spanish', 3) 323
('de', 3) 280
('french', 3) 269
('–', 3) 249
('king', 3) 345
('ottoman', 3) 293
('war', 

In [66]:
save_obj(frecuencias, "Frecuencias")

In [67]:
frecuencias= load_obj("Frecuencias")

In [19]:
def etiquetas(palabras,frecuencias,clases):
    clases = 5 
    import numpy as np
    x = np.zeros((1, clases + 1))
    for palabra in palabras:
        for t in range(1, clases + 1):
            x[0,t] += frecuencias.get((palabra, t),0)
    return x

In [20]:
clases = 5
cont = 0
for i in textos.index:
    cont += 1
    if cont > 1000000:
        break
    palabras = []    
    for word in textos["TextoLimpio"][i][:-1].split(","):
        word = word[2:-1]
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        palabras.append(word)    
        for k in range(1, clases + 1):
            textos["Clase" + str(k)][i] = etiquetas(palabras,frecuencias,clases)[0,k]
    clear_output( wait = True )
    print(f"Creando Features {i} de {len(textos.index)}") 
textos

Creando Features 5865 de 5866


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",5031.0,4266.0,3941.0,2614.0,3024.0,NaN,1,757,78
1,"During his youth, Alexander was tutored by Ari...","['youth', 'alexand', 'tutor', 'aristotl', 'age...",4970.0,4769.0,3691.0,2864.0,2354.0,NaN,1,821,74
2,"Alexander endeavoured to reach the ""ends of th...","['alexand', 'endeavour', 'reach', 'end', 'worl...",3438.0,3330.0,3299.0,3899.0,1949.0,NaN,1,624,58
3,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",6087.0,5257.0,3469.0,3093.0,4105.0,NaN,1,1007,87
4,"Alexander was born in Pella, the capital of th...","['alexand', 'born', 'pella', 'capit', 'kingdom...",2764.0,2296.0,2605.0,1068.0,1246.0,NaN,1,488,46
...,...,...,...,...,...,...,...,...,...,...,...
5861,Some agriculturalists also regularly hunt and ...,"['agriculturalist', 'also', 'regularli', 'hunt...",1832.0,1877.0,2027.0,1653.0,3124.0,NaN,5,592,54
5862,There are nevertheless a number of contemporar...,"['nevertheless', 'number', 'contemporari', 'hu...",2941.0,3362.0,3080.0,2456.0,4209.0,NaN,5,1011,86
5863,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",2400.0,2324.0,3101.0,2506.0,7782.0,NaN,5,549,54
5864,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'would', 'eventu', 'flourish...",2222.0,2320.0,2447.0,2439.0,4630.0,NaN,5,724,71


In [21]:
textos.to_csv("Historia.csv")

In [22]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",5031.0,4266.0,3941.0,2614.0,3024.0,NaN,1,757,78
1,"During his youth, Alexander was tutored by Ari...","['youth', 'alexand', 'tutor', 'aristotl', 'age...",4970.0,4769.0,3691.0,2864.0,2354.0,NaN,1,821,74
2,"Alexander endeavoured to reach the ""ends of th...","['alexand', 'endeavour', 'reach', 'end', 'worl...",3438.0,3330.0,3299.0,3899.0,1949.0,NaN,1,624,58
3,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",6087.0,5257.0,3469.0,3093.0,4105.0,NaN,1,1007,87
4,"Alexander was born in Pella, the capital of th...","['alexand', 'born', 'pella', 'capit', 'kingdom...",2764.0,2296.0,2605.0,1068.0,1246.0,NaN,1,488,46
...,...,...,...,...,...,...,...,...,...,...,...
5861,Some agriculturalists also regularly hunt and ...,"['agriculturalist', 'also', 'regularli', 'hunt...",1832.0,1877.0,2027.0,1653.0,3124.0,NaN,5,592,54
5862,There are nevertheless a number of contemporar...,"['nevertheless', 'number', 'contemporari', 'hu...",2941.0,3362.0,3080.0,2456.0,4209.0,NaN,5,1011,86
5863,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",2400.0,2324.0,3101.0,2506.0,7782.0,NaN,5,549,54
5864,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'would', 'eventu', 'flourish...",2222.0,2320.0,2447.0,2439.0,4630.0,NaN,5,724,71


In [24]:
textos.sample(10)

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
465,Modern archaeology in formerly Punic areas has...,"['modern', 'archaeolog', 'formerli', 'punic', ...",4201.0,3533.0,3062.0,2428.0,6595.0,NaN,1,1039,95
3778,A 2017 National Bureau of Economic Research pa...,"['2017', 'nation', 'bureau', 'econom', 'resear...",2123.0,3119.0,2208.0,2161.0,3249.0,NaN,4,634,58
4090,"About an hour later, when Ferdinand was return...","['hour', 'later', 'ferdinand', 'return', 'visi...",2167.0,2574.0,2901.0,2615.0,1710.0,NaN,4,1015,95
1641,Charlemagne encouraged clerics to translate Ch...,"['charlemagn', 'encourag', 'cleric', 'translat...",3254.0,4636.0,3738.0,2401.0,3085.0,NaN,2,1046,94
1525,"Under Charlemagne, the Saxon Wars had as their...","['charlemagn', 'saxon', 'war', 'chief', 'objec...",1568.0,2514.0,1633.0,2113.0,992.0,NaN,2,432,41
733,"Archaic Roman religion, at least concerning th...","['archaic', 'roman', 'religion', 'least', 'con...",6677.0,4669.0,3677.0,2322.0,2416.0,NaN,1,945,90
519,"Following the death of Ramesses XI in 1078 BC,...","['follow', 'death', 'ramess', 'xi', '1078bc', ...",6770.0,6536.0,5569.0,4529.0,5670.0,NaN,1,1325,129
12,"Suda writes that, also, Anaximenes of Lampsacu...","['suda', 'write', 'also', 'anaximen', 'lampsac...",884.0,778.0,705.0,501.0,731.0,NaN,1,128,11
1574,"From 781 (Pallars, Ribagorça) to 806 (Pamplona...","['781', 'pallar', 'ribagorça', '806', 'pamplon...",1631.0,3414.0,2251.0,1669.0,915.0,NaN,2,905,86
3226,"The Arg-e Bam, in Kerman Province in southeast...","['arg-', 'bam', 'kerman', 'provinc', 'southeas...",2861.0,2192.0,2043.0,1774.0,3233.0,NaN,3,633,65


In [26]:
textos.groupby(by=["CLASE"]).count()

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,LTCrudo,LTLimpio
CLASE,,,,,,,,,,
1,1165,1165,1165,1165,1165,1165,1165,0,1165,1165
2,1274,1274,1274,1274,1274,1274,1274,0,1274,1274
3,1206,1206,1206,1206,1206,1206,1206,0,1206,1206
4,1028,1028,1028,1028,1028,1028,1028,0,1028,1028
5,1193,1193,1193,1193,1193,1193,1193,0,1193,1193


In [27]:
textos.sample(10)

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
5579,"In the 1980s, important archaeological discove...","['1980', 'import', 'archaeolog', 'discoveri', ...",406.0,333.0,365.0,219.0,662.0,NaN,5,175,18
5707,Little interest was shown by white settlers in...,"['littl', 'interest', 'shown', 'white', 'settl...",2545.0,2510.0,2468.0,2355.0,3019.0,NaN,5,512,49
2294,"At his death, the role of the emperor as a pat...","['death', 'role', 'emperor', 'patron', 'easter...",2360.0,3049.0,2396.0,2112.0,946.0,NaN,2,558,56
5202,"Subsequently, British scholars used either Eva...","['subsequ', 'british', 'scholar', 'use', 'eith...",4741.0,4846.0,4332.0,3686.0,9657.0,NaN,5,1143,104
3513,"Initially on good terms with William and Mary,...","['initi', 'good', 'term', 'william', 'mari', '...",543.0,962.0,993.0,944.0,581.0,NaN,3,304,24
5809,Current artists in vogue include Jacinta Hayes...,"['current', 'artist', 'vogu', 'includ', 'jacin...",322.0,431.0,415.0,292.0,577.0,NaN,5,268,29
2767,The geography of North Africa has been reasona...,"['geographi', 'north', 'africa', 'reason', 'we...",1164.0,1063.0,1247.0,749.0,1489.0,NaN,3,238,24
2557,The English naval commander Francis Drake sail...,"['english', 'naval', 'command', 'franci', 'dra...",2044.0,2815.0,4228.0,2094.0,2588.0,NaN,3,768,87
614,The Roman poet Virgil recounted this legend in...,"['roman', 'poet', 'virgil', 'recount', 'legend...",2830.0,2331.0,2293.0,2122.0,1121.0,NaN,1,494,49
2270,"In the later part of his reign, John focused h...","['later', 'part', 'reign', 'john', 'focus', 'a...",3488.0,5984.0,3910.0,4260.0,1856.0,NaN,2,947,87


In [46]:
# Obtenemos una fila de features
columna = 2
print("CLASE",textos.loc[614,"CLASE"])
CLASE = textos.loc[614,"CLASE"]
print(textos.iloc[614,columna:columna + clases].values)
print("Máximo feature",np.argmax(textos.iloc[614,columna:columna + clases].values) + 1)
featureMax = np.argmax(textos.iloc[614,columna:columna + clases].values) + 1
CLASE == featureMax

CLASE 1
[2830.0 2331.0 2293.0 2122.0 1121.0]
Máximo feature 1


True

In [58]:
cont = 0
for i in textos.index:
    cont += 1
    if cont > 1000000:
        break
    print(textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1)
    textos["Predicción"][i] = textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1
    clear_output( wait = True )
    print(f"Comparando CLASE y features {i} de {len(textos.index)}") 
textos        

Comparando CLASE y features 5865 de 5866


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",5031.0,4266.0,3941.0,2614.0,3024.0,True,1,757,78
1,"During his youth, Alexander was tutored by Ari...","['youth', 'alexand', 'tutor', 'aristotl', 'age...",4970.0,4769.0,3691.0,2864.0,2354.0,True,1,821,74
2,"Alexander endeavoured to reach the ""ends of th...","['alexand', 'endeavour', 'reach', 'end', 'worl...",3438.0,3330.0,3299.0,3899.0,1949.0,False,1,624,58
3,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",6087.0,5257.0,3469.0,3093.0,4105.0,True,1,1007,87
4,"Alexander was born in Pella, the capital of th...","['alexand', 'born', 'pella', 'capit', 'kingdom...",2764.0,2296.0,2605.0,1068.0,1246.0,True,1,488,46
...,...,...,...,...,...,...,...,...,...,...,...
5861,Some agriculturalists also regularly hunt and ...,"['agriculturalist', 'also', 'regularli', 'hunt...",1832.0,1877.0,2027.0,1653.0,3124.0,True,5,592,54
5862,There are nevertheless a number of contemporar...,"['nevertheless', 'number', 'contemporari', 'hu...",2941.0,3362.0,3080.0,2456.0,4209.0,True,5,1011,86
5863,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",2400.0,2324.0,3101.0,2506.0,7782.0,True,5,549,54
5864,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'would', 'eventu', 'flourish...",2222.0,2320.0,2447.0,2439.0,4630.0,True,5,724,71


In [59]:
textos.to_csv("Historia.csv")

In [60]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",5031.0,4266.0,3941.0,2614.0,3024.0,True,1,757,78
1,"During his youth, Alexander was tutored by Ari...","['youth', 'alexand', 'tutor', 'aristotl', 'age...",4970.0,4769.0,3691.0,2864.0,2354.0,True,1,821,74
2,"Alexander endeavoured to reach the ""ends of th...","['alexand', 'endeavour', 'reach', 'end', 'worl...",3438.0,3330.0,3299.0,3899.0,1949.0,False,1,624,58
3,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",6087.0,5257.0,3469.0,3093.0,4105.0,True,1,1007,87
4,"Alexander was born in Pella, the capital of th...","['alexand', 'born', 'pella', 'capit', 'kingdom...",2764.0,2296.0,2605.0,1068.0,1246.0,True,1,488,46
...,...,...,...,...,...,...,...,...,...,...,...
5861,Some agriculturalists also regularly hunt and ...,"['agriculturalist', 'also', 'regularli', 'hunt...",1832.0,1877.0,2027.0,1653.0,3124.0,True,5,592,54
5862,There are nevertheless a number of contemporar...,"['nevertheless', 'number', 'contemporari', 'hu...",2941.0,3362.0,3080.0,2456.0,4209.0,True,5,1011,86
5863,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",2400.0,2324.0,3101.0,2506.0,7782.0,True,5,549,54
5864,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'would', 'eventu', 'flourish...",2222.0,2320.0,2447.0,2439.0,4630.0,True,5,724,71


In [61]:
textos.groupby(by=["Predicción"]).count()

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,CLASE,LTCrudo,LTLimpio
Predicción,,,,,,,,,,
False,1104,1104,1104,1104,1104,1104,1104,1104,1104,1104
True,4762,4762,4762,4762,4762,4762,4762,4762,4762,4762


In [63]:
rate = 100 * 4762 / (1104 + 4762)
print("Porcentaje de features acertados",rate)

Porcentaje de features acertados 81.17967950903511


## Ahora vamos a pasarlo por un modelo
Modelo 20.81

## Logramos un accuracy del 85%

### Vamos a extraer la lista de palabras confusas que habíamos obtenido antes 

In [68]:
confusas = load_obj("Confusas")

In [69]:
len(confusas)

1213

In [74]:
vocabulario = load_obj("Vocabulario")

In [70]:
confusas

['interven',
 'multipl',
 'stepp',
 'basi',
 'sold',
 'obtain',
 'permit',
 '49',
 'timber',
 'pull',
 'fact',
 '...',
 'half',
 'compos',
 'travel',
 'facilit',
 'exert',
 'head',
 '183',
 'erect',
 'center',
 'alon',
 'innov',
 'villag',
 'princip',
 'militarili',
 'save',
 'inform',
 'broken',
 'coastal',
 'ceas',
 '22',
 'flight',
 '138',
 'commonli',
 'late',
 'basic',
 'cousin',
 'overland',
 '95',
 'cours',
 '212',
 'accompani',
 'kilometr',
 'function',
 '96',
 'scholarship',
 'upon',
 '197',
 'intent',
 'fifth',
 'model',
 '200',
 'outcom',
 '24',
 '123',
 '7',
 'decad',
 'confin',
 '–',
 'laid',
 'deal',
 'mani',
 'forward',
 'balanc',
 'devic',
 'spirit',
 'legaci',
 'thereaft',
 'life',
 'two',
 'first',
 'amount',
 '157',
 'greatest',
 'scheme',
 '27',
 'nearbi',
 'intellig',
 'creation',
 'comput',
 'young',
 'contribut',
 'done',
 'lead',
 'examin',
 'reliabl',
 'involv',
 'kill',
 'except',
 '51',
 'fulli',
 '216',
 'root',
 'neighbor',
 'enabl',
 'symbol',
 'conclus',


In [71]:
textos.sample(10)

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
173,It was ideal to mine stones from quarries that...,"['ideal', 'mine', 'stone', 'quarri', 'situat',...",3302.0,2460.0,2183.0,2143.0,4789.0,False,1,732,70
5330,Their ultimate extinction coincides with Heinr...,"['ultim', 'extinct', 'coincid', 'heinrich', 'e...",5091.0,5021.0,5444.0,4222.0,13312.0,True,5,1217,120
3425,"During this time, the Portuguese gradually swi...","['time', 'portugues', 'gradual', 'switch', 'in...",2046.0,2729.0,3954.0,2352.0,2569.0,True,3,662,59
1443,Physical anthropologists have found striking s...,"['physic', 'anthropologist', 'found', 'strike'...",531.0,880.0,440.0,250.0,1296.0,False,2,233,18
5189,"The world's oldest known engineered roadway, t...","[""world'"", 'oldest', 'known', 'engin', 'roadwa...",534.0,539.0,542.0,425.0,891.0,True,5,190,19
176,Cranes were used for construction work and pos...,"['crane', 'use', 'construct', 'work', 'possibl...",1699.0,1492.0,1615.0,1242.0,2325.0,False,1,337,33
4452,The 1945 Allied conference in San Francisco es...,"['1945', 'alli', 'confer', 'san', 'francisco',...",752.0,873.0,962.0,2112.0,489.0,True,4,455,40
4849,Petroglyphs appeared in the Neolithic. A Petro...,"['petroglyph', 'appear', 'neolith', 'petroglyp...",1280.0,1428.0,1553.0,1405.0,2690.0,True,5,468,47
3658,"Originally set up as law courts, by the mid-18...","['origin', 'set', 'law', 'court', 'mid', '18th...",1257.0,1825.0,1897.0,1704.0,863.0,False,4,484,45
1449,The Italian language preserves a large number ...,"['italian', 'languag', 'preserv', 'larg', 'num...",1001.0,2095.0,1225.0,1584.0,828.0,True,2,429,39


In [77]:
cont = 0
for i in textos.index:
    cont += 1
    if cont > 1000000:
        break
    
    print()
    print("Antes de sacar las confusas",len(textos["TextoLimpio"][i][:-1].split(",")),textos["TextoLimpio"][i][:-1].split(","))     
    print()
        
    palabras = []    
    for word in textos["TextoLimpio"][i][:-1].split(","):
        word = word[2:-1]
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        assert (word in vocabulario),"La palabra " + word + "no está en el vocabulario!"   
        if word not in confusas: 
            palabras.append(word)    

    print()
    print("SIN confusas",len(palabras),palabras)     
    print()
            
    textos["TextoLimpio"][i] = palabras        
            
    clear_output( wait = True )
    print(f"Sacando confusas {i} de {len(textos.index)}") 
textos    

Saando confusas 5865 de 5866


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,"[alexand, iii, macedon, greek, αλέξανδρος, γʹ,...",5031.0,4266.0,3941.0,2614.0,3024.0,True,1,757,78
1,"During his youth, Alexander was tutored by Ari...","[alexand, tutor, aristotl, age, philip, assass...",4970.0,4769.0,3691.0,2864.0,2354.0,True,1,821,74
2,"Alexander endeavoured to reach the ""ends of th...","[alexand, endeavour, sea, invad, india, 326bc,...",3438.0,3330.0,3299.0,3899.0,1949.0,False,1,624,58
3,Alexander's legacy includes the cultural diffu...,"[alexander, cultur, diffus, syncret, conquest,...",6087.0,5257.0,3469.0,3093.0,4105.0,True,1,1007,87
4,"Alexander was born in Pella, the capital of th...","[alexand, born, pella, capit, kingdom, macedon...",2764.0,2296.0,2605.0,1068.0,1246.0,True,1,488,46
...,...,...,...,...,...,...,...,...,...,...,...
5861,Some agriculturalists also regularly hunt and ...,"[agriculturalist, hunt, gather, farm, frost-fr...",1832.0,1877.0,2027.0,1653.0,3124.0,True,5,592,54
5862,There are nevertheless a number of contemporar...,"[nevertheless, hunter-gather, contact, influen...",2941.0,3362.0,3080.0,2456.0,4209.0,True,5,1011,86
5863,Evidence suggests big-game hunter-gatherers cr...,"[evid, suggest, big-gam, hunter-gather, bere, ...",2400.0,2324.0,3101.0,2506.0,7782.0,True,5,549,54
5864,Hunter-gatherers would eventually flourish all...,"[hunter-gather, america, base, plain, unit, st...",2222.0,2320.0,2447.0,2439.0,4630.0,True,5,724,71


In [79]:
for i in textos.index:
    textos["LTCrudo"][i]  = len(textos["TextoCrudo"][i])
    textos["LTLimpio"][i] = len(textos["TextoLimpio"][i])
    clear_output( wait = True )
    print(f"Calculando logitudes {i} de {len(textos.index)}") 
textos

Calculando logitudes 5865 de 5866


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,"[alexand, iii, macedon, greek, αλέξανδρος, γʹ,...",5031.0,4266.0,3941.0,2614.0,3024.0,True,1,757,59
1,"During his youth, Alexander was tutored by Ari...","[alexand, tutor, aristotl, age, philip, assass...",4970.0,4769.0,3691.0,2864.0,2354.0,True,1,821,53
2,"Alexander endeavoured to reach the ""ends of th...","[alexand, endeavour, sea, invad, india, 326bc,...",3438.0,3330.0,3299.0,3899.0,1949.0,False,1,624,38
3,Alexander's legacy includes the cultural diffu...,"[alexander, cultur, diffus, syncret, conquest,...",6087.0,5257.0,3469.0,3093.0,4105.0,True,1,1007,56
4,"Alexander was born in Pella, the capital of th...","[alexand, born, pella, capit, kingdom, macedon...",2764.0,2296.0,2605.0,1068.0,1246.0,True,1,488,36
...,...,...,...,...,...,...,...,...,...,...,...
5861,Some agriculturalists also regularly hunt and ...,"[agriculturalist, hunt, gather, farm, frost-fr...",1832.0,1877.0,2027.0,1653.0,3124.0,True,5,592,31
5862,There are nevertheless a number of contemporar...,"[nevertheless, hunter-gather, contact, influen...",2941.0,3362.0,3080.0,2456.0,4209.0,True,5,1011,47
5863,Evidence suggests big-game hunter-gatherers cr...,"[evid, suggest, big-gam, hunter-gather, bere, ...",2400.0,2324.0,3101.0,2506.0,7782.0,True,5,549,41
5864,Hunter-gatherers would eventually flourish all...,"[hunter-gather, america, base, plain, unit, st...",2222.0,2320.0,2447.0,2439.0,4630.0,True,5,724,42


# Vamos a recalcular las frecuencias sin confusas 

In [81]:
frecuencias = defaultdict(int)
clases = 5
cont = 0
for i in textos.index:
    cont += 1
    if cont > 10000000:
        break
    for word in textos["TextoLimpio"][i][:-1]:
        # word = word[2:-1]
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        frecuencias[(word, textos["CLASE"][i])] = frecuencias.get((word, textos["CLASE"][i]), 0) + 1    
    clear_output( wait = True )
    print(f"Creando Frecuencias {i} de {len(textos.index)}")      
frecuencias

Creando Frecuencias 5865 de 5866


defaultdict(int,
            {('alexand', 1): 292,
             ('iii', 1): 56,
             ('macedon', 1): 18,
             ('greek', 1): 270,
             ('αλέξανδρος', 1): 1,
             ('γʹ', 1): 1,
             ('ὁ', 1): 2,
             ('μακεδών', 1): 1,
             ('aléxandro', 1): 1,
             ('ho', 1): 2,
             ('makedȏn', 1): 1,
             ('20/21', 1): 1,
             ('juli', 1): 7,
             ('356bc', 1): 3,
             ('10/11', 1): 1,
             ('june', 1): 3,
             ('323bc', 1): 4,
             ('μέγας', 1): 1,
             ('méga', 1): 1,
             ('king', 1): 237,
             ('basileu', 1): 3,
             ('ancient', 1): 213,
             ('kingdom', 1): 124,
             ('argead', 1): 1,
             ('dynasti', 1): 81,
             ('born', 1): 15,
             ('pella', 1): 3,
             ('succeed', 1): 22,
             ('father', 1): 48,
             ('philip', 1): 56,
             ('ii', 1): 159,
             ('throne', 

In [83]:
clases = 5
cont = 0
for i in textos.index:
    cont += 1
    if cont > 1000000:
        break
    palabras = []    
    for word in textos["TextoLimpio"][i][:-1]:
        # word = word[2:-1]
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        palabras.append(word)    
        for k in range(1, clases + 1):
            textos["Clase" + str(k)][i] = etiquetas(palabras,frecuencias,clases)[0,k]
    clear_output( wait = True )
    print(f"Creando Features {i} de {len(textos.index)}") 
textos

Creando Features 5865 de 5866


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,"[alexand, iii, macedon, greek, αλέξανδρος, γʹ,...",4178.0,3318.0,2902.0,1619.0,1906.0,True,1,757,59
1,"During his youth, Alexander was tutored by Ari...","[alexand, tutor, aristotl, age, philip, assass...",4551.0,4340.0,3038.0,2170.0,1284.0,True,1,821,53
2,"Alexander endeavoured to reach the ""ends of th...","[alexand, endeavour, sea, invad, india, 326bc,...",2466.0,2370.0,2153.0,2834.0,1095.0,False,1,624,38
3,Alexander's legacy includes the cultural diffu...,"[alexander, cultur, diffus, syncret, conquest,...",4719.0,3547.0,1722.0,1280.0,2471.0,True,1,1007,56
4,"Alexander was born in Pella, the capital of th...","[alexand, born, pella, capit, kingdom, macedon...",2216.0,1909.0,2229.0,589.0,560.0,True,1,488,36
...,...,...,...,...,...,...,...,...,...,...,...
5861,Some agriculturalists also regularly hunt and ...,"[agriculturalist, hunt, gather, farm, frost-fr...",581.0,574.0,722.0,589.0,1467.0,True,5,592,31
5862,There are nevertheless a number of contemporar...,"[nevertheless, hunter-gather, contact, influen...",890.0,1215.0,989.0,726.0,1769.0,True,5,1011,47
5863,Evidence suggests big-game hunter-gatherers cr...,"[evid, suggest, big-gam, hunter-gather, bere, ...",1176.0,1116.0,1720.0,1603.0,6468.0,True,5,549,41
5864,Hunter-gatherers would eventually flourish all...,"[hunter-gather, america, base, plain, unit, st...",994.0,1056.0,1116.0,1356.0,2915.0,True,5,724,42


In [84]:
save_obj(frecuencias, "Frecuencias sin Confusas")

In [86]:
clases = 5
cont = 0
for i in textos.index:
    cont += 1
    if cont > 1000000:
        break
    palabras = []    
    for word in textos["TextoLimpio"][i][:-1]:
        # word = word[2:-1]
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        assert (word in vocabulario),"La palabra " + word + "no está en el vocabulario!"      
        palabras.append(word)    
        for k in range(1, clases + 1):
            textos["Clase" + str(k)][i] = etiquetas(palabras,frecuencias,clases)[0,k]
    clear_output( wait = True )
    print(f"Creando Features {i} de {len(textos.index)}") 
textos

Creando Features 5865 de 5866


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,"[alexand, iii, macedon, greek, αλέξανδρος, γʹ,...",4178.0,3318.0,2902.0,1619.0,1906.0,True,1,757,59
1,"During his youth, Alexander was tutored by Ari...","[alexand, tutor, aristotl, age, philip, assass...",4551.0,4340.0,3038.0,2170.0,1284.0,True,1,821,53
2,"Alexander endeavoured to reach the ""ends of th...","[alexand, endeavour, sea, invad, india, 326bc,...",2466.0,2370.0,2153.0,2834.0,1095.0,False,1,624,38
3,Alexander's legacy includes the cultural diffu...,"[alexander, cultur, diffus, syncret, conquest,...",4719.0,3547.0,1722.0,1280.0,2471.0,True,1,1007,56
4,"Alexander was born in Pella, the capital of th...","[alexand, born, pella, capit, kingdom, macedon...",2216.0,1909.0,2229.0,589.0,560.0,True,1,488,36
...,...,...,...,...,...,...,...,...,...,...,...
5861,Some agriculturalists also regularly hunt and ...,"[agriculturalist, hunt, gather, farm, frost-fr...",581.0,574.0,722.0,589.0,1467.0,True,5,592,31
5862,There are nevertheless a number of contemporar...,"[nevertheless, hunter-gather, contact, influen...",890.0,1215.0,989.0,726.0,1769.0,True,5,1011,47
5863,Evidence suggests big-game hunter-gatherers cr...,"[evid, suggest, big-gam, hunter-gather, bere, ...",1176.0,1116.0,1720.0,1603.0,6468.0,True,5,549,41
5864,Hunter-gatherers would eventually flourish all...,"[hunter-gather, america, base, plain, unit, st...",994.0,1056.0,1116.0,1356.0,2915.0,True,5,724,42


In [87]:
textos.to_csv("Historia.csv")

In [88]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",4178.0,3318.0,2902.0,1619.0,1906.0,True,1,757,59
1,"During his youth, Alexander was tutored by Ari...","['alexand', 'tutor', 'aristotl', 'age', 'phili...",4551.0,4340.0,3038.0,2170.0,1284.0,True,1,821,53
2,"Alexander endeavoured to reach the ""ends of th...","['alexand', 'endeavour', 'sea', 'invad', 'indi...",2466.0,2370.0,2153.0,2834.0,1095.0,False,1,624,38
3,Alexander's legacy includes the cultural diffu...,"['alexander', 'cultur', 'diffus', 'syncret', '...",4719.0,3547.0,1722.0,1280.0,2471.0,True,1,1007,56
4,"Alexander was born in Pella, the capital of th...","['alexand', 'born', 'pella', 'capit', 'kingdom...",2216.0,1909.0,2229.0,589.0,560.0,True,1,488,36
...,...,...,...,...,...,...,...,...,...,...,...
5861,Some agriculturalists also regularly hunt and ...,"['agriculturalist', 'hunt', 'gather', 'farm', ...",581.0,574.0,722.0,589.0,1467.0,True,5,592,31
5862,There are nevertheless a number of contemporar...,"['nevertheless', 'hunter-gather', 'contact', '...",890.0,1215.0,989.0,726.0,1769.0,True,5,1011,47
5863,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",1176.0,1116.0,1720.0,1603.0,6468.0,True,5,549,41
5864,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'america', 'base', 'plain', ...",994.0,1056.0,1116.0,1356.0,2915.0,True,5,724,42


In [89]:
textos.sample(10)

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
2410,"Islam came to the Southeast Asia, first by the...","['islam', 'came', 'southeast', 'muslim', 'trad...",636.0,1856.0,729.0,490.0,744.0,True,2,573,29
560,The earliest preserved ancient Egyptian temple...,"['earliest', 'ancient', 'egyptian', 'templ', '...",2347.0,1606.0,1051.0,610.0,1897.0,True,1,880,57
4761,A modern-day diet known as the Paleolithic die...,"['modern-day', 'diet', 'paleolith', 'diet', 'b...",270.0,211.0,255.0,114.0,1763.0,True,5,206,14
4899,The Middle Mumun pottery period culture of the...,"['middl', 'mumun', 'potteri', 'period', 'cultu...",1583.0,1201.0,687.0,374.0,3556.0,True,5,567,41
3255,"In 1556, amid great pomp, and leaning on the s...","['1556', 'amid', 'pomp', 'lean', 'shoulder', '...",2599.0,3142.0,3652.0,1137.0,338.0,True,3,596,46
1726,The term feudal has also been applied to non-W...,"['feudal', 'non-western', 'institut', 'attitud...",962.0,2012.0,990.0,1016.0,843.0,True,2,681,36
3073,"Unfortunately for Anne, her partial victory de...","['unfortun', 'ann', 'victori', 'condé', 'want'...",866.0,1541.0,3788.0,1660.0,432.0,True,3,822,65
3780,The Jacobin cause was picked up by Marxists in...,"['jacobin', 'pick', 'marxist', 'centuri', 'ele...",341.0,574.0,273.0,767.0,154.0,True,4,197,12
2603,"Great advances in science have been termed ""re...","['scienc', 'revolut', '18th', 'centuri', '1747...",856.0,1453.0,1683.0,1596.0,1234.0,True,3,614,35
1139,... a very witty remark is reported to have be...,"['witti', 'wife', 'argentocoxu', 'caledonian',...",1046.0,490.0,484.0,488.0,156.0,True,1,477,29


In [90]:
cont = 0
for i in textos.index:
    cont += 1
    if cont > 1000000:
        break
    print(textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1)
    textos["Predicción"][i] = textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1
    clear_output( wait = True )
    print(f"Comparando CLASE y features {i} de {len(textos.index)}") 
textos      

Comparando CLASE y features 5865 de 5866


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",4178.0,3318.0,2902.0,1619.0,1906.0,True,1,757,59
1,"During his youth, Alexander was tutored by Ari...","['alexand', 'tutor', 'aristotl', 'age', 'phili...",4551.0,4340.0,3038.0,2170.0,1284.0,True,1,821,53
2,"Alexander endeavoured to reach the ""ends of th...","['alexand', 'endeavour', 'sea', 'invad', 'indi...",2466.0,2370.0,2153.0,2834.0,1095.0,False,1,624,38
3,Alexander's legacy includes the cultural diffu...,"['alexander', 'cultur', 'diffus', 'syncret', '...",4719.0,3547.0,1722.0,1280.0,2471.0,True,1,1007,56
4,"Alexander was born in Pella, the capital of th...","['alexand', 'born', 'pella', 'capit', 'kingdom...",2216.0,1909.0,2229.0,589.0,560.0,False,1,488,36
...,...,...,...,...,...,...,...,...,...,...,...
5861,Some agriculturalists also regularly hunt and ...,"['agriculturalist', 'hunt', 'gather', 'farm', ...",581.0,574.0,722.0,589.0,1467.0,True,5,592,31
5862,There are nevertheless a number of contemporar...,"['nevertheless', 'hunter-gather', 'contact', '...",890.0,1215.0,989.0,726.0,1769.0,True,5,1011,47
5863,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",1176.0,1116.0,1720.0,1603.0,6468.0,True,5,549,41
5864,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'america', 'base', 'plain', ...",994.0,1056.0,1116.0,1356.0,2915.0,True,5,724,42


In [94]:
textos.sample(10)

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
2268,Alexios's son John II Komnenos succeeded him i...,"['alexios', 'son', 'john', 'ii', 'komneno', 's...",1456.0,2689.0,1397.0,692.0,366.0,True,2,450,31
1893,The Anglo-Norman dynasty that had ruled Englan...,"['anglo-norman', 'dynasti', 'rule', 'england',...",2340.0,5642.0,4365.0,2110.0,338.0,True,2,609,45
3929,The peace with Britain proved to be uneasy and...,"['peac', 'britain', 'uneasi', 'controversi', '...",779.0,798.0,1104.0,2314.0,176.0,True,4,479,33
2554,The 1497 English expedition led by Italian Ven...,"['1497', 'english', 'expedit', 'led', 'italian...",1740.0,3173.0,4644.0,2155.0,2115.0,True,3,976,65
2964,The picture of Elizabeth painted by her Protes...,"['elizabeth', 'paint', 'protest', 'admir', '17...",1844.0,2570.0,2906.0,2536.0,1519.0,True,3,776,54
3632,A common view is that European combat methods ...,"['european', 'combat', 'militari', 'tactic', '...",2150.0,2984.0,4043.0,5806.0,1019.0,False,3,842,52
4110,Neither side proved able to deliver a decisive...,"['neither', 'decis', 'year', '1915', '17', 'br...",693.0,1158.0,1174.0,2113.0,737.0,True,4,372,21
5654,An initial agreement to fund restoration was a...,"['fund', 'restor', 'unit', 'nation', 'scientif...",318.0,341.0,292.0,662.0,460.0,False,5,236,11
4157,"On 3 May 1917, during the Nivelle Offensive, t...","['3', '1917', 'nivel', 'offens', 'french', 'co...",991.0,1846.0,2334.0,3609.0,977.0,True,4,741,50
597,The golden ratio seems to be reflected in many...,"['golden', 'ratio', 'reflect', 'egyptian', 'co...",804.0,177.0,159.0,72.0,219.0,True,1,274,17


In [105]:
textos.to_csv("Historia.csv")

In [96]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",4178.0,3318.0,2902.0,1619.0,1906.0,True,1,757,59
1,"During his youth, Alexander was tutored by Ari...","['alexand', 'tutor', 'aristotl', 'age', 'phili...",4551.0,4340.0,3038.0,2170.0,1284.0,True,1,821,53
2,"Alexander endeavoured to reach the ""ends of th...","['alexand', 'endeavour', 'sea', 'invad', 'indi...",2466.0,2370.0,2153.0,2834.0,1095.0,False,1,624,38
3,Alexander's legacy includes the cultural diffu...,"['alexander', 'cultur', 'diffus', 'syncret', '...",4719.0,3547.0,1722.0,1280.0,2471.0,True,1,1007,56
4,"Alexander was born in Pella, the capital of th...","['alexand', 'born', 'pella', 'capit', 'kingdom...",2216.0,1909.0,2229.0,589.0,560.0,False,1,488,36
...,...,...,...,...,...,...,...,...,...,...,...
5861,Some agriculturalists also regularly hunt and ...,"['agriculturalist', 'hunt', 'gather', 'farm', ...",581.0,574.0,722.0,589.0,1467.0,True,5,592,31
5862,There are nevertheless a number of contemporar...,"['nevertheless', 'hunter-gather', 'contact', '...",890.0,1215.0,989.0,726.0,1769.0,True,5,1011,47
5863,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",1176.0,1116.0,1720.0,1603.0,6468.0,True,5,549,41
5864,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'america', 'base', 'plain', ...",994.0,1056.0,1116.0,1356.0,2915.0,True,5,724,42


In [97]:
textos.groupby(by=["CLASE"]).count()

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,LTCrudo,LTLimpio
CLASE,,,,,,,,,,
1,1165,1165,1165,1165,1165,1165,1165,1165,1165,1165
2,1274,1274,1274,1274,1274,1274,1274,1274,1274,1274
3,1206,1206,1206,1206,1206,1206,1206,1206,1206,1206
4,1028,1028,1028,1028,1028,1028,1028,1028,1028,1028
5,1193,1193,1193,1193,1193,1193,1193,1193,1193,1193


In [98]:
textos.groupby(by=["Predicción"]).count()

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,CLASE,LTCrudo,LTLimpio
Predicción,,,,,,,,,,
False,1049,1049,1049,1049,1049,1049,1049,1049,1049,1049
True,4817,4817,4817,4817,4817,4817,4817,4817,4817,4817


In [100]:
rate = 100 * 4817 / (1049 + 4762)
print("Porcentaje de features acertados",rate)

Porcentaje de features acertados 82.89451041128893


**Cómo último paso de esta notebook vamos a pasarlo por un modelo de ANN a ver que accuracy nos    da. En la próxima notebook vamos a pasar en limpio esta funciones y sistematizarlo creando métodos en la clase Textos** 

In [101]:
len(vocabulario)

26756

In [102]:
len(frecuencias)

41381

In [103]:
len(confusas)

1213

In [104]:
save_obj(vocabulario, "Vocabulario")
save_obj(frecuencias, "Frecuencias")
save_obj(confusas, "confusas")

**Logramos un accuracy del 86% al entrenar el modelo con una ANN FC. La lista de palabras confusas extraídas fue obtenida con el dataset anterior. Resta generar una lista de palabras confusas para este dataset, recalcular las frecuencias y reentrenar**